# When Explanations Lie: Why Modified BP Attribution fails

This notebook produces the heatmap figures and the ssim results.

In [ ]:
# uncomment to install install packages
#!pip install tensorflow-gpu==1.13.1
#!pip install innvestigate seaborn tqdm

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf

import innvestigate
import matplotlib.pyplot as plt

import numpy as np
import PIL 
import copy
import contextlib

import imp
import numpy as np
import os

from skimage.measure import compare_ssim 
import pickle
import datetime
from collections import OrderedDict
from IPython.display import IFrame, display

import keras
import keras.backend
import keras.models


import innvestigate
import innvestigate.applications.imagenet
import innvestigate.utils as iutils
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.analyzer.relevance_based.relevance_analyzer import LRP
from innvestigate.analyzer.base import AnalyzerNetworkBase, ReverseAnalyzerBase
from innvestigate.analyzer.deeptaylor import DeepTaylor

import time
import tqdm

import seaborn as sns

import itertools
import matplotlib as mpl

from tensorflow.python.client import device_lib

from when_explanations_lie import *
from monkey_patch import custom_add_bn_rule, get_custom_rule

In [ ]:
#def _prepare_model(self, model):
#    return super(DeepTaylor, self)._prepare_model(model)
#
## otherwise DTD does not work on negative outputs
#DeepTaylor._prepare_model = _prepare_model

In [ ]:
# device_lib.list_local_devices()

In [ ]:
# path to imagenet validation
# imagenet_val_dir = "/home/leonsixt/tmp/imagenet/imagenet-raw/validation"
imagenet_val_dir = "/mnt/ssd/data/imagenet/imagenet-raw/validation"
# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
n_selected_imgs = 10

model_names = ['resnet50', 'vgg16']

assert os.path.exists(imagenet_val_dir)
os.makedirs('figures', exist_ok=True)

In [ ]:
model, innv_net, color_conversion = load_model('vgg16')
ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

keras.backend.clear_session()
model, innv_net, color_conversion = load_model('resnet50')
ex_image, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)


assert ((ex_image - ex_image_vgg) == 0).all()
print("Loaded {} images".format(n_selected_imgs))

nice_layer_names = get_nice_layer_names(model)

In [ ]:
n_layers = {'vgg16': 22, 'resnet50': 177}

randomization_layers = {
    'vgg16': ["conv1_1", "conv2_1",  "conv3_1", "conv4_1",  "conv4_3",  "conv5_1", "conv5_3", "fc1", "fc3"],
    'resnet50': ['conv1', 'block2_2', 'block3_1', 'block3_3', 'block4_1', 'block4_6', 'block5_2', 'dense'],
}

In [ ]:
output_shapes = get_output_shapes(model)

print_output_shapes = False 
if print_output_shapes: 
    print("{:3}{:20}{:20}{}".format("l", "layer", "input_at_0", "output_shape"))
    for i in range(len(model.layers)):
        layer = model.get_layer(index=i)
        print("{:3}: {:20}  {:20}  {}".format(
            i, layer.name, str(layer.get_input_shape_at(0)), str(output_shapes[i])))
        #print("{:3}: {:20}  {:20}  {}".format(i, type(layer).__name__, layer.name, output_shapes[i]))

In [ ]:
def hmap_postprocess_wrapper(name):
    return lambda x: heatmap_postprocess(name, x)

input_range = (ex_image.min(), ex_image.max())
analysers = get_analyser_params(input_range)

attr_names = [n for (n, _, _, _, _) in analysers]
    
hmap_postprocessing = {
    n: hmap_postprocess_wrapper(post_name) for n, _, post_name, _, _ in analysers
}

In [ ]:
for i, name in enumerate(attr_names):
    style = mpl_styles[name]
    plt.plot(np.arange(10), [20-i] * 10, 
             #markersize=5,
             label=name + " m=" + style['marker'], **style)
    
plt.legend(bbox_to_anchor=(1, 1))

## Sanity Checks: Random Parameters & Logit Check

In [ ]:
model_cascading, _, _ = load_model('resnet50')
model_random, _, _ = load_model('resnet50', load_weights=False)
model_cascading.set_weights(model.get_weights())
out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print("mean-l1 distance of the outputs of the trained model and when weights are from trained model [should be 0]:", np.abs(out_cascading - out).mean())

n_layers = len(model_random.layers)
copy_weights(model_cascading, model_random, range(n_layers - 3, n_layers))

out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print( "mean-l1 distance of the outputs of the trained model when the last 2 layers are random [should not be 0]:", np.abs(out_cascading - out).mean())

In [ ]:
if 'hmap_output_dir' in globals():
    raise Exception()
hmap_output_dir = 'cache/' + datetime.datetime.now().isoformat()
os.makedirs(hmap_output_dir)

In [ ]:
print(hmap_output_dir)

In [ ]:
# heatmaps are saved in those dicts

recreate_analyser = False
for model_name in tqdm.tqdm_notebook(model_names):
    get_layer_idx = lambda layer_name: get_layer_idx_full(
        model_name, nice_layer_names, layer_name)
    
    for i, (attr_name, innv_name, _, excludes, analyser_kwargs) in enumerate(tqdm.tqdm_notebook(
        analysers, desc=model_name)):
        
        hmap_original = OrderedDict()
        hmap_random_weights = OrderedDict()
        hmap_random_target = OrderedDict()
        
        if i % 1 == 0:
            # clear session from time to time to not OOM
            keras.backend.clear_session()

            model, innv_net, _ = load_model(model_name)
            model_cascading, _, _ = load_model(model_name)
            model_random, _, _ = load_model(model_name, load_weights=False)
            model_cascading.set_weights(model.get_weights())
            
        if "exclude_" + model_name in excludes:
            continue
        if innv_name == 'pattern.attribution':
            analyser_kwargs['patterns'] = innv_net['patterns']

        cascading_heatmaps = {}
        cascading_outputs = {}
        model_cascading.set_weights(model.get_weights())

        original_idx = len(model.layers)
        custom_rule = get_custom_rule(innv_name, analyser_kwargs)
        with custom_add_bn_rule(custom_rule):
            analyzer_cascading = innvestigate.create_analyzer(
                innv_name, model_cascading, 
                neuron_selection_mode="index", **analyser_kwargs)

            for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
                random_target = get_random_target(target)
                hmap_random_target[model_name, attr_name, img_idx] = (
                    random_target, analyzer_cascading.analyze(img_pp, neuron_selection=random_target)[0])
            selected_layers = [('original', original_idx)] +  [
                (name, get_layer_idx(name)) 
                 for name in randomization_layers[model_name][::-1]
            ]
            for layer_name, layer_idx in tqdm.tqdm_notebook(selected_layers, desc=attr_name):
                copy_weights(model_cascading, model_random, range(layer_idx, original_idx))
                if recreate_analyser:
                    analyzer_cascading = create_analyzer(
                        analyser, model_cascading, 
                        neuron_selection_mode="index",  **analyser_kwargs)

                for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
                    hmap = analyzer_cascading.analyze(img_pp, neuron_selection=target)[0]
                    if layer_idx == original_idx:
                        hmap_original[model_name, attr_name, img_idx] = hmap
                    else:
                        hmap_random_weights[model_name, attr_name, img_idx, layer_idx] =  hmap


            with open(hmap_output_dir + '/heatmap_{}_{}.pickle'.format(model_name, attr_name), 'wb') as f:
                pickle.dump((hmap_original, hmap_random_weights, hmap_random_target), f)

In [ ]:
dump_heatmaps = False

if dump_heatmaps:
    os.makedirs('cache', exist_ok=True)
    with open('cache/heatmaps_v4.pickle', 'wb') as f:
        pickle.dump((hmap_original, hmap_random_weights, hmap_random_target, analysers), f)

In [ ]:
# print size of dump
! ls -lh 'heatmaps.pickle'

In [ ]:
load_heatmaps = False
if load_heatmaps:
    with open('heatmaps.pickle', 'rb') as f:
        hmap_original, hmap_random_weights, hmap_random_target, analysers = pickle.load(f)

In [ ]:
#attr_names = sorted(set([n for (n, _) in hmap_original.keys()]))
attr_names

In [ ]:
import warnings

In [ ]:
ssim_random_weights = OrderedDict()
l2_random_weights = OrderedDict()

last_idx = len(model.layers)
for model_name in model_names:
    for attr_name in attr_names:
        outpath = hmap_output_dir + '/heatmap_{}_{}.pickle'.format(model_name, attr_name)
        if not os.path.exists(outpath):
            warnings.warn("not found: " + outpath)
            continue
        with open(outpath, 'rb') as f:
            hmap_original, hmap_random_weights, hmap_random_target = pickle.load(f)
        
        for (model_name, name, img_idx, layer_idx), heatmap in tqdm.tqdm_notebook(
            hmap_random_weights.items(), desc="{}.{}".format(model_name, attr_name)):
            original_heatmap = hmap_original[model_name, name, img_idx]
            postprocess = hmap_postprocessing[name]
            original_heatmap = postprocess(original_heatmap)
            heatmap = postprocess(heatmap)
            ssim_random_weights[model_name, name, img_idx, layer_idx] = ssim_flipped(heatmap, original_heatmap)
            l2_random_weights[model_name, name, img_idx, layer_idx] = l2_flipped(heatmap, original_heatmap)

In [ ]:
ssim_random_target = OrderedDict()

for model_name in model_names:
    for attr_name in attr_names:
        outpath = hmap_output_dir + 'cache/heatmap_{}_{}.pickle'.format(model_name, attr_name)
        if not os.path.exists(outpath):
            warnings.warn("not found: " + outpath)
            continue
        with open(outpath, 'rb') as f:
            hmap_original, hmap_random_weights, hmap_random_target = pickle.load(f)
        for (model_name, attr_name, img_idx), (_, hmap_random) in tqdm.tqdm_notebook(
            hmap_random_target.items()):
            if (model_name, attr_name) not in ssim_random_target:
                ssim_random_target[model_name, attr_name] = []

            postprocess = hmap_postprocessing[attr_name]

            hmap = hmap_original[model_name, attr_name, img_idx]
            original_heatmap = postprocess(original_heatmap)
            hmap = postprocess(hmap)
            hmap_random = postprocess(hmap_random)
            ssim_random_target[model_name, attr_name].append(
                ssim_flipped(hmap, hmap_random))

In [ ]:
ssim_random_target.keys()

In [ ]:
for model_name in model_names:
    with sns.axes_style('ticks', {"axes.grid": True, 'font.family': 'serif'}):
        fig, ax = plt.subplots(1, 1, figsize=(4, 3.0), squeeze=True)


        xlabels =  [n for (m, n) in ssim_random_target.keys() if m == model_name]
        bars = ax.boxplot([ssim_random_target[model_name, n] for n in xlabels]) 
        ax.set_ylabel('SSIM')
        #ax.set_xticks(np.arange(len(xlabels)))
        ax.set_xticklabels(xlabels, rotation=90)
        ax.set_ylim(-0.05, 1.05)
        
        os.makedirs('figures', exist_ok=True)
        figpath = 'figures/random-logit-boxplot-{}.pdf'.format(model_name)
        fig.savefig(figpath,  bbox_inches='tight', pad_inches=0)
        display(IFrame(figpath, 800, 500))

In [ ]:
original_idx

In [ ]:

ssim_reduce = 'median'
confidence_intervals = True
confidence_percentile = 99.5

for model_name in model_names:
    
    selected_layers = [
        (name, get_layer_idx_full(model_name, nice_layer_names, name)) 
         for name in randomization_layers[model_name][::-1]
    ]
    
    with sns.axes_style("ticks", {"axes.grid": True, 'font.family': 'serif'}):
        # metrics = [('SSIM', ssim), ('MSE', l2_random_weights)]
        metrics = [('SSIM', ssim_random_weights)]
        fig, axes = plt.subplots(1, len(metrics), figsize=(4.5 * len(metrics), 3.5), squeeze=False)
        axes = axes[0]
        for ax, (ylabel, metric) in zip(axes, metrics): 
            for (name, _, _, excludes, _) in analysers:
                if 'exclude_' + model_name in excludes:
                    continue
                    
                metric_per_layer = []

                lower_conf = []
                upper_conf = []
                for (_, layer_idx) in selected_layers[::-1]:
                    metric_per_layer.append(
                        [metric[model_name, name, img_idx, layer_idx] for img_idx in selected_img_idxs]
                    )
                    if confidence_intervals:
                        vals = np.array(metric_per_layer[-1])
                        ridx = np.random.choice(len(vals), (10000, len(vals)), replace=True)
                        resample = vals[ridx]
                        stats = np.median(resample, 1)
                        lower_conf.append(np.percentile(stats, 100 - confidence_percentile))
                        upper_conf.append(np.percentile(stats, confidence_percentile))

                metric_per_layer = np.array(metric_per_layer)

                if ssim_reduce == 'mean':
                    ssims_reduced = metric_per_layer.mean(1)
                elif ssim_reduce == 'median':
                    ssims_reduced = np.median(metric_per_layer, 1)

                ticks = np.arange(len(ssims_reduced))
                ax.plot(ticks, ssims_reduced[::-1], label=name, **mpl_styles[name])
                ax.fill_between(ticks, lower_conf[::-1], upper_conf[::-1], 
                                color=mpl_styles[name]['color'],
                                alpha=0.25
                               )
                #ax.plot(ticks, lower_conf, color=linestyles[name]['color'])
                #ax.plot(ticks, upper_conf, color=linestyles[name]['color'])

            xlabels = [layer_name for layer_name, _ in selected_layers] 
            ax.set_ylim([0, 1.05])
            ax.set_xticks(np.arange(len(xlabels)))
            ax.set_xticklabels(xlabels, rotation=90)
            ax.set_ylabel(ylabel)
        axes[-1].legend(bbox_to_anchor=(1.0, 1.00))
        plt.savefig('ssim-random-weights-{}.pdf'.format(model_name),  bbox_inches='tight', pad_inches=0)
        plt.show()
        display(IFrame('ssim-random-weights-{}.pdf'.format(model_name), 800, 500))

In [ ]:
def plot_heatmap_grid(heatmaps, cols, row_labels=[], column_labels=[], fig_path=None, figsize=None):
    mpl.rcParams['font.family'] = 'serif'
    rows = len(heatmaps) // cols
    
    if figsize is None:
        figsize = (cols, rows)
    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False)
    fontsize = 9
    plt.subplots_adjust(wspace=0.05, hspace=0.05, top=1, bottom=0, left=0, right=1)
    for label, ax in zip(row_labels, axes[:, 0]):
        ax.set_ylabel(label, fontsize=fontsize + 1, labelpad=55, rotation=0)
        
    print(axes.shape, column_labels, row_labels)
    for label, ax in zip(column_labels, axes[0, :]):
        ax.set_title(label, fontsize=fontsize)
        
        
    for ax, heatmap in zip(axes.flatten(), heatmaps):
        ax.imshow(heatmap, cmap='seismic', vmin=-1, vmax=1)
        ax.set_xticks([])
        ax.set_yticks([])

    #plt.tight_layout()
    if fig_path is not None:
        plt.savefig(fig_path, bbox_inches='tight', pad_inches=0, dpi=120)

In [ ]:
def normalize_neg(x):
    vmax = np.percentile(x, 99)
    vmin = np.percentile(x, 1)
    vmax
    x_pos = x * (x > 0)
    x_neg = x * (x < 0)
    
    x_pos = x_pos / vmax
    x_neg = - x_neg / vmin
    return np.clip(x_pos + x_neg, -1, 1)


def load_examplary_heatmap():
    hmap_loaded = []
    for model_name in model_names:
        rnd_layers = randomization_layers[model_name][::-1]
        for (attr_name, _, _, excludes, _) in tqdm.tqdm_notebook(analysers):
            if 'exclude_' + model_name in excludes:
                print(attr_name)
                continue
            try:
                outpath = hmap_output_dir + '/heatmap_{}_{}.pickle'.format(model_name, attr_name)
                if not os.path.exists(outpath):
                    warnings.warn("not found: " + outpath)
                    continue
                with open(outpath, 'rb') as f:
                    hmap_original, hmap_random_weights, hmap_random_target = pickle.load(f)
                if attr_name in ['GuidedBP', 'Deconv']:
                    postp = hmap_postprocess_wrapper('sum')
                else:
                    postp = hmap_postprocessing[attr_name]

                for img_idx in selected_img_idxs[:1]:
                    hmap_loaded.append((model_name, attr_name, 'image', 
                                        norm_image(val_images[0][0][0])))
                    hmap_loaded.append((model_name, attr_name, 'original', 
                                        normalize_neg(postp(hmap_original[model_name, attr_name, img_idx]))))
                    for layer_name in rnd_layers:
                        layer_idx = get_layer_idx_full(model_name, nice_layer_names, layer_name)
                        hmap_loaded.append((model_name, attr_name, layer_name, 
                                            normalize_neg(postp(hmap_random_weights[model_name, attr_name, img_idx, layer_idx]))))
            except KeyError as e:
                print(e)
                pass
    return hmap_loaded

hmap_examplary = load_examplary_heatmap()

In [ ]:

outpath = hmap_output_dir + '/heatmap_{}_{}.pickle'.format('resnet50', 
                                                           'LRP-$\\alpha=1, \\beta=0$' )
print(outpath)
with open(outpath, 'rb') as f:
    hmap_original, hmap_random_weights, hmap_random_target = pickle.load(f)

In [ ]:
for (_, _, img_idx, layer_idx), hmap in hmap_random_weights.items():
    print(img_idx, (hmap==0).all())

In [ ]:
[n for (n, _, _, _, _) in analysers]

In [ ]:
os.makedirs('figures/sanity_checks', exist_ok=True)

In [ ]:
# figure 1
selected_analysers = ['GuidedBP',
 'Deconv',
 'DTD',
 'LRP-$\\alpha=1, \\beta=0$',
 'LRP-$\\alpha=2, \\beta=1$',
 'PatternAttr.',
 'Gradient'
]
selected_layers = [
    "image",
    "original",
    "fc3",
    "conv5_3",
    "conv4_1",
    "conv2_1",
    "conv1_1",
]

selected_layers
selected_hmaps = []

for (model_name, attr_name, layer_name, hmap) in hmap_examplary:
    if model_name != 'vgg16':
        continue
    if attr_name not in selected_analysers:
        continue
    if layer_name not in selected_layers:
        continue
       
    selected_hmaps.append(hmap)
    
plot_heatmap_grid(
    selected_hmaps, len(selected_layers), row_labels=selected_analysers, 
    column_labels=selected_layers,
    figsize=(3.9, 0.55*len(selected_analysers) + 0.1),
    fig_path='figures/sanity_checks/heatmap_grid_figure1.pdf'
)

In [ ]:
display(IFrame('figures/sanity_checks/heatmap_grid_figure1.pdf', width=1000, height=600))

In [ ]:
selected_analysers = [
 'LRP-cmp-$\\alpha=1$',
 'LRP-cmp-$\\alpha=2$',
]


for selected_model_name in model_names:
    
    selected_hmaps = []
    selected_layers = {
        'vgg16': [
            "image",
            "original",
            "fc3",
            "conv5_3",
            "conv4_1",
            "conv2_1",
            "conv1_1",
        ],
        'resnet50': [
            "image",
            "original",
            "dense",
            "block5_2",
            "block3_3",
            "block2_2",
            "conv1",
        ]}[selected_model_name]
    
    for (model_name, attr_name, layer_name, hmap) in hmap_examplary:
        if model_name != selected_model_name:
            continue
        if attr_name not in selected_analysers:
            continue
        if layer_name not in selected_layers:
            print(layer_name)
            continue

        selected_hmaps.append(hmap)

    outname = 'figures/sanity_checks/heatmap_grid_{}_lrp_cmp.pdf'.format(selected_model_name)
    plot_heatmap_grid(
        selected_hmaps, len(selected_layers), row_labels=selected_analysers, 
        column_labels=selected_layers,
        figsize=(3.99, 1.1),
        fig_path=outname
    )
    display(IFrame(outname, width=1000, height=600))

In [ ]:
for selected_model_name in model_names:
    print(model_name)
    attr_for_model = OrderedDict()
    layer_names = OrderedDict()
    hmap_plot = []
    for (model_name, attr_name, layer_name, hmap) in hmap_examplary:
        if model_name != selected_model_name:
            continue
        attr_for_model[attr_name] = attr_name
        layer_names[layer_name] = layer_name
        hmap_plot.append(hmap)
        
    plot_heatmap_grid(
        hmap_plot, len(layer_names), 
        row_labels=attr_for_model.keys(), 
        column_labels=layer_names.keys(),
        figsize=(0.6*len(layer_names), 0.6*len(attr_for_model)),
        fig_path='figures/sanity_checks/heatmap_grid_{}.pdf'.format(selected_model_name)
    )

In [ ]:
display(IFrame('figures/sanity_checks/heatmap_grid_vgg16.pdf', width=1000, height=600))
display(IFrame('figures/sanity_checks/heatmap_grid_resnet50.pdf', width=1000, height=600))

In [ ]:
! ls 'figures/sanity_checks/'

In [ ]:
attr_name = 'LRP-$\\alpha=5, \\beta=4$'
attr_name = 'PatternAttr.'
hmap = hmap_random_weights['vgg16', attr_name, 673, 1]